In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm

def clean_data(df):
    # Handle missing values and placeholders
    df = df.replace(["None", "N/A"], np.nan)

    # Handle -9999.0 values in numeric columns
    num_cols = df.select_dtypes(include=['number']).columns
    df[num_cols] = df[num_cols].replace(-9999.0, np.nan)

    # Convert timestamp columns
    time_cols = [col for col in ["id4", "id5", "id12", "id13", "f370"] if col in df.columns]
    for col in time_cols:
        df[col] = pd.to_datetime(df[col], errors='coerce')

    # Convert key columns to string for consistent merging
    for col in ["id2", "id3"]:
        if col in df.columns:
            df[col] = df[col].astype(str)

    return df

def create_features(df, events, transactions, offers):
    # Ensure merge keys are strings
    df["id2"] = df["id2"].astype(str)
    df["id3"] = df["id3"].astype(str)
    events["id2"] = events["id2"].astype(str)
    events["id3"] = events["id3"].astype(str)
    transactions["id2"] = transactions["id2"].astype(str)
    offers["id3"] = offers["id3"].astype(str)

    # --- 1. Precompute Aggregates ---

    # Customer-level transaction features
    customer_trans = transactions.groupby('id2').agg(
        avg_trans_amount=('f367', 'mean'),
        max_trans_amount=('f367', 'max'),
        last_trans_date=('f370', 'max'),
        trans_count=('f370', 'count')
    ).reset_index()

    # Customer-level event features
    customer_events = events.groupby('id2').agg(
        total_impressions=('id4', 'count'),
        total_clicks=('id7', lambda x: x.notna().sum()),
        last_impression=('id4', 'max')
    ).reset_index()
    customer_events['cust_click_rate'] = customer_events['total_clicks'] / customer_events['total_impressions']

    # Offer-level features
    offer_features = offers.groupby('id3').agg(
        discount_rate=('f376', 'mean'),
        offer_duration=('id13', lambda x: (x.max() - x.min()).days)
    ).reset_index()

    # Customer-offer interaction features
    interaction_features = events.groupby(['id2', 'id3']).agg(
        past_impressions=('id4', 'count'),
        past_clicks=('id7', lambda x: x.notna().sum()),
        last_interaction=('id4', 'max')
    ).reset_index()
    interaction_features['past_click_rate'] = interaction_features['past_clicks'] / interaction_features['past_impressions']

    # --- 2. Merge Features ---
    df = df.merge(customer_trans, on='id2', how='left')
    df = df.merge(customer_events, on='id2', how='left')
    df = df.merge(offer_features, on='id3', how='left')
    df = df.merge(interaction_features, on=['id2', 'id3'], how='left')

    # --- 3. Temporal Features ---
    df['impression_hour'] = df['id4'].dt.hour
    df['impression_dow'] = df['id4'].dt.dayofweek
    df['is_weekend'] = df['impression_dow'].isin([5, 6]).astype(int)

    df['days_since_last_trans'] = (df['id4'] - df['last_trans_date']).dt.days
    df['days_since_last_interaction'] = (df['id4'] - df['last_interaction']).dt.days

    # --- 4. Derived Features ---
    df['discount_sensitivity'] = df['discount_rate'] * df['cust_click_rate']
    df['spend_discount_affinity'] = df['avg_trans_amount'] * df['discount_rate']

    # --- 5. Fill Missing Values ---
    num_cols = ['avg_trans_amount', 'cust_click_rate', 'discount_rate', 
                'past_click_rate', 'days_since_last_trans']
    for col in num_cols:
        df[col] = df[col].fillna(df[col].median())

    count_cols = ['past_impressions', 'past_clicks', 'total_impressions']
    for col in count_cols:
        df[col] = df[col].fillna(0)

    date_cols = ['last_trans_date', 'last_interaction']
    old_date = pd.Timestamp('2000-01-01')
    for col in date_cols:
        df[col] = df[col].fillna(old_date)

    return df

def process_data(train_path, test_path, events_path, transactions_path, offers_path):
    print("Loading data...")
    train = pd.read_parquet(train_path)
    test = pd.read_parquet(test_path)
    events = pd.read_parquet(events_path)
    transactions = pd.read_parquet(transactions_path)
    offers = pd.read_parquet(offers_path)

    print("Cleaning data...")
    train = clean_data(train)
    test = clean_data(test)
    events = clean_data(events)
    transactions = clean_data(transactions)
    offers = clean_data(offers)

    print("Preprocessing events...")
    events['click_time'] = events['id7']
    events['has_clicked'] = events['click_time'].notna().astype(int)

    print("Creating features for train set...")
    train = create_features(train, events, transactions, offers)

    print("Creating features for test set...")
    test = create_features(test, events, transactions, offers)

    print("Finalizing datasets...")
    common_cols = list(set(train.columns) & set(test.columns))
    train = train[common_cols + ['y']]
    test = test[common_cols]

    return train, test

if __name__ == "__main__":
    train, test = process_data(
        train_path="/kaggle/input/amex-problem1/train_data.parquet",
        test_path="/kaggle/input/amex-problem1/test_data.parquet",
        events_path="/kaggle/input/amex-problem1/add_event.parquet",
        transactions_path="/kaggle/input/amex-problem1/add_trans.parquet",
        offers_path="/kaggle/input/amex-problem1/offer_metadata.parquet"
    )


In [9]:
import pandas as pd
import numpy as np
from tqdm import tqdm

def clean_data(df):
    # Handle missing values and placeholders
    df = df.replace(["None", "N/A"], np.nan)

    # Handle -9999.0 values in numeric columns
    num_cols = df.select_dtypes(include=['number']).columns
    df[num_cols] = df[num_cols].replace(-9999.0, np.nan)

    # Convert timestamp columns
    time_cols = [col for col in ["id4", "id5", "id12", "id13", "f370"] if col in df.columns]
    for col in time_cols:
        df[col] = pd.to_datetime(df[col], errors='coerce')

    # Convert key columns to string for consistent merging
    for col in ["id2", "id3"]:
        if col in df.columns:
            df[col] = df[col].astype(str)

    return df

def create_features(df, events, transactions, offers):
    # Ensure merge keys are strings
    df["id2"] = df["id2"].astype(str)
    df["id3"] = df["id3"].astype(str)
    events["id2"] = events["id2"].astype(str)
    events["id3"] = events["id3"].astype(str)
    transactions["id2"] = transactions["id2"].astype(str)
    offers["id3"] = offers["id3"].astype(str)

    # --- 1. Precompute Aggregates ---

    # Customer-level transaction features
    customer_trans = transactions.groupby('id2').agg(
        avg_trans_amount=('f367', 'mean'),
        max_trans_amount=('f367', 'max'),
        last_trans_date=('f370', 'max'),
        trans_count=('f370', 'count'),
        recency_days=('f370', lambda x: (pd.Timestamp('2025-07-02') - x.max()).days),
        frequency_30d=('f370', lambda x: x.gt(pd.Timestamp('2025-06-02')).sum())
    ).reset_index()

    # Customer-level event features
    customer_events = events.groupby('id2').agg(
        total_impressions=('id4', 'count'),
        total_clicks=('id7', lambda x: x.notna().sum()),
        last_impression=('id4', 'max')
    ).reset_index()
    customer_events['cust_click_rate'] = customer_events['total_clicks'] / customer_events['total_impressions']

    # Offer-level features
    offer_features = offers.groupby('id3').agg(
        discount_rate=('f376', 'mean'),
        offer_duration=('id13', lambda x: (x.max() - x.min()).days),
        avg_offer_ctr=('f377', 'mean') if 'f377' in offers.columns else pd.Series(np.nan)
    ).reset_index()

    # Customer-offer interaction features
    interaction_features = events.groupby(['id2', 'id3']).agg(
        past_impressions=('id4', 'count'),
        past_clicks=('id7', lambda x: x.notna().sum()),
        last_interaction=('id4', 'max')
    ).reset_index()
    interaction_features['past_click_rate'] = interaction_features['past_clicks'] / interaction_features['past_impressions']

    # --- 2. Merge Features ---
    df = df.merge(customer_trans, on='id2', how='left')
    df = df.merge(customer_events, on='id2', how='left')
    df = df.merge(offer_features, on='id3', how='left')
    df = df.merge(interaction_features, on=['id2', 'id3'], how='left')

    # --- 3. Temporal Features ---
    df['impression_hour'] = df['id4'].dt.hour
    df['impression_dow'] = df['id4'].dt.dayofweek
    df['is_weekend'] = df['impression_dow'].isin([5, 6]).astype(int)

    df['days_since_last_trans'] = (df['id4'] - df['last_trans_date']).dt.days
    df['days_since_last_interaction'] = (df['id4'] - df['last_interaction']).dt.days

    # --- 4. Derived Features ---
    df['discount_sensitivity'] = df['discount_rate'] * df['cust_click_rate']
    df['spend_discount_affinity'] = df['avg_trans_amount'] * df['discount_rate']

    # --- 5. Additional Features ---
    # Recency, Frequency, Monetary (RFM) features
    df['recency_days'] = df['recency_days'].fillna(df['recency_days'].median())
    df['frequency_30d'] = df['frequency_30d'].fillna(0)
    df['monetary_value'] = df['avg_trans_amount'] * df['trans_count']

    # Fill missing values
    num_cols = ['avg_trans_amount', 'cust_click_rate', 'discount_rate', 
                'past_click_rate', 'days_since_last_trans', 'recency_days', 'frequency_30d', 'monetary_value']
    for col in num_cols:
        df[col] = df[col].fillna(df[col].median())

    count_cols = ['past_impressions', 'past_clicks', 'total_impressions']
    for col in count_cols:
        df[col] = df[col].fillna(0)

    date_cols = ['last_trans_date', 'last_interaction']
    old_date = pd.Timestamp('2000-01-01')
    for col in date_cols:
        df[col] = df[col].fillna(old_date)

    return df


def process_data(train_path, test_path, events_path, transactions_path, offers_path):
    print("Loading data...")
    train = pd.read_parquet(train_path)
    test = pd.read_parquet(test_path)
    events = pd.read_parquet(events_path)
    transactions = pd.read_parquet(transactions_path)
    offers = pd.read_parquet(offers_path)

    print("Cleaning data...")
    train = clean_data(train)
    test = clean_data(test)
    events = clean_data(events)
    transactions = clean_data(transactions)
    offers = clean_data(offers)

    print("Preprocessing events...")
    events['click_time'] = events['id7']
    events['has_clicked'] = events['click_time'].notna().astype(int)

    print("Creating features for train set...")
    train = create_features(train, events, transactions, offers)

    print("Creating features for test set...")
    test = create_features(test, events, transactions, offers)

    print("Finalizing datasets...")
    common_cols = list(set(train.columns) & set(test.columns))
    train = train[common_cols + ['y']]
    test = test[common_cols]

    return train, test

if __name__ == "__main__":
    train, test = process_data(
        train_path="/kaggle/input/amex-problem1/train_data.parquet",
        test_path="/kaggle/input/amex-problem1/test_data.parquet",
        events_path="/kaggle/input/amex-problem1/add_event.parquet",
        transactions_path="/kaggle/input/amex-problem1/add_trans.parquet",
        offers_path="/kaggle/input/amex-problem1/offer_metadata.parquet")

Loading data...
Cleaning data...
Preprocessing events...
Creating features for train set...
Creating features for test set...
Finalizing datasets...


In [11]:
# import pandas as pd
import numpy as np
from sklearn.model_selection import TimeSeriesSplit
from sklearn.utils import shuffle
import lightgbm as lgb
from lightgbm import early_stopping, log_evaluation
from tqdm import tqdm

# === Custom MAP@7 ===
def calculate_map7(y_true, y_pred, id2_val, k=7):
    df = pd.DataFrame({'id2': id2_val, 'true': y_true, 'pred': y_pred})
    df['rank'] = df.groupby('id2')['pred'].rank(method='first', ascending=False)
    top_k = df[df['rank'] <= k].copy()
    top_k['correct'] = top_k['true'].astype(bool)
    top_k['cumulative_correct'] = top_k.groupby('id2')['correct'].cumsum()
    top_k['precision_at_k'] = top_k['cumulative_correct'] / top_k['rank']
    ap_per_customer = top_k.groupby('id2')['precision_at_k'] \
        .apply(lambda x: x[x > 0].mean() if any(x > 0) else 0)
    return ap_per_customer.mean()

# === Data Prep ===
X = train.drop(['y', 'id1', 'id4', 'id5'], axis=1)
y = train['y'].astype(int)
id2_series = train['id2'].astype(str)

X = X.apply(pd.to_numeric, errors='coerce').fillna(np.nan)
X = X.select_dtypes(include='number')

# === Final Fold Validation ===
tscv = TimeSeriesSplit(n_splits=5)
splits = list(tscv.split(X))
train_idx, val_idx = splits[-1]

X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
id2_val = id2_series.iloc[val_idx].values

# === Shuffle ===
X_train, y_train = shuffle(X_train, y_train, random_state=42)

# === LightGBM ===
lgb_model = lgb.LGBMClassifier(
    objective='binary',
    learning_rate=0.03,
    num_leaves=128,
    max_depth=9,
    subsample=0.85,
    colsample_bytree=0.85,
    reg_alpha=0.3,
    reg_lambda=0.3,
    n_estimators=5000,
    random_state=42
)
lgb_model.fit(
    X_train, y_train,
    eval_set=[(X_val, y_val)],
    eval_metric='binary_logloss',
    callbacks=[
        early_stopping(stopping_rounds=20),
        log_evaluation(100)
    ]
)
best_iter_lgb = lgb_model.best_iteration_
val_pred_lgb = lgb_model.predict_proba(X_val, num_iteration=best_iter_lgb)[:, 1]
print("MAP@7 LGB:", calculate_map7(y_val, val_pred_lgb, id2_val))

[LightGBM] [Info] Number of positive: 31192, number of negative: 610612
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 2.131444 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 48004
[LightGBM] [Info] Number of data points in the train set: 641804, number of used features: 359
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.048601 -> initscore=-2.974300
[LightGBM] [Info] Start training from score -2.974300
Training until validation scores don't improve for 20 rounds
[100]	valid_0's binary_logloss: 0.106565
[200]	valid_0's binary_logloss: 0.105927
Early stopping, best iteration is:
[180]	valid_0's binary_logloss: 0.105848
MAP@7 LGB: 0.05499165820366656
MAP@7 CatBoost: 0.0556849168357849


/usr/local/lib/python3.11/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


MAP@7 XGBoost: 0.05568497637447911


In [12]:
lgb_params = lgb_model.get_params()
lgb_params['n_estimators'] = best_iter_lgb

lgb_model_final = lgb.LGBMClassifier(**lgb_params)
lgb_model_final.fit(X, y)

cat_params = cat_model.get_params()
cat_params['iterations'] = cat_model.get_best_iteration()

cat_model_final = CatBoostClassifier(**cat_params)
cat_model_final.fit(X, y, verbose=0)

xgb_params = xgb_model.get_xgb_params()
xgb_params['n_estimators'] = best_iter_xgb

xgb_model_final = xgb.XGBClassifier(**xgb_params)
xgb_model_final.fit(X, y)


[LightGBM] [Info] Number of positive: 37051, number of negative: 733113
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 1.808288 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 48272
[LightGBM] [Info] Number of data points in the train set: 770164, number of used features: 359
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.048108 -> initscore=-2.985005
[LightGBM] [Info] Start training from score -2.985005


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.85, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.03, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=9,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=154,
              n_jobs=None, num_parallel_tree=None, random_state=42, ...)

In [13]:
# === Final Predictions ===
X_test = test[X.columns].apply(pd.to_numeric, errors='coerce').fillna(np.nan)
lgb_preds = lgb_model_final.predict_proba(X_test)[:, 1]
cat_preds = cat_model_final.predict_proba(X_test)[:, 1]
xgb_preds = xgb_model_final.predict_proba(X_test)[:, 1]

test_preds = (lgb_preds + cat_preds + xgb_preds) / 3
test['pred'] = test_preds + 1e-6 * np.random.rand(len(test))  # tie-breaking noise

# === Submission File ===
required_cols = ['id1', 'id2', 'id3', 'id5', 'pred']
for col in ['id1', 'id2', 'id3', 'id5']:
    test[col] = test[col].astype(str).str.strip()

submission = test[required_cols].copy()
submission.to_csv("r2_final_submission_ensemble.csv", index=False)
print("✅ Final submission shape:", submission.shape) # should be (369301,5)


✅ Final submission shape: (369301, 5)
